# Object-Oriented Programming with Classes
*Caveat: There will be code duplication as classes are built upon to improve functionality. When calling from the REPL, we would need to import the module prior to instantiating.*

### Class Definition
- A class is used to define the structure and behavior of one or more objects
- Each object defined by the class is referred to as an instance of the class
- The class of an object controls its initialization and which attributes and methods are avaialble

In [1]:
# create an empty class
# class names are written in CamelCase
class Flight:
    pass

# create a new object (instance) by calling the class constructor
f = Flight()
print(f)
print(type(f))

<class '__main__.Flight'>


### Instance Methods
- Methods are just functions defined within the class block
- Instance methods are functions which can be called on objects which are instances of our class
- Instance methods must accept a reference to the instance on which the method was called as the first formal argument
- By convention, the first formal argument is *self*

In [2]:
class Flight:

    def flight_number(self):
        return "SN060"

f = Flight()
f.flight_number()

'SN060'

### Instance Initializers
- If provided, the initializer method is called as part of the process of creating a new object when we call the constructor
- The initializer method must be called \_\_init\_\_() delimited by the double underscores (dunder method)
- The first argument to the initializer method must be *self*
- The initializer method should not return anything, it modifies the object referred to by *self*
- Within the initializer we assign to an attribute of the newly created instance called _flight_number
- The leading underscore is used to avoid name clash with the method and as a convention for objects not intended to be consumed or manipulated

In [3]:
class Flight:

    def __init__(self, flight_number):
        self._flight_number = flight_number
    
    def flight_number(self):
        return self._flight_number

f = Flight(flight_number = "SN060")
f.flight_number()

'SN060'

### Validation and Invariants
- It's good practice for the initializer of an object to establish class invariants
- Invariants are truths about the objects of the class that should endure for the lifetime of the object
- We establish class invariants in the \_\_init\_\_() method and raise exceptions if they can't be attained

In [4]:
class Flight:

    def __init__(self, flight_number):
        if not flight_number[:2].isalpha():
            raise ValueError(f"No Airline Code in {flight_number}")
        
        if not flight_number[:2].isupper():
            raise ValueError(f"Invalid Airline Code {flight_number}")
        
        if not (flight_number[2:].isdigit() and int(flight_number[2:]) <= 9999):
            raise ValueError(f"Invalid Route Number {flight_number}")
        
        self._flight_number = flight_number

    def flight_number(self):
        return self._flight_number
    
    def airline(self):
        return self._flight_number[:2]

In [5]:
# testing first check (airline code must be two letters)
# results in ValueError: No Airline Code in 060
# Flight("060")

In [6]:
# testing second check (airline code must be upper case)
# results in ValueError: Invalid Airline Code sn060
# Flight("sn060")

In [7]:
# testing third check (preceeding characters are numeric and less than or equal to 9999)
# results in ValueError: Invalid Route Number SN12345
# Flight('SN12345') 

In [8]:
# flight number passes checks and is returned by flight_number method
# airline code is returned by airline method
f = Flight('SN060')

print(
    f"""
    Airline Code: {f.airline()}
    Flight Number: {f.flight_number()}
    """
)


    Airline Code: SN
    Flight Number: SN060
    


### Second Class
- Model different kinds of aircraft
- The initializer creates four attributes for the aircraft (registration, model, num rows, num seats per row)
- The seating_plan method is added to return the allowed rows and seats as a series of tuples

The call to the range() constructor produces a range object which can be used as an iterable series of row numbers, up to the number of rows in the plane plus one. We add one to the range object because the stopping point of a range is not inclusive. The string and its slice method return a string with one character per seat. These two objects – the range and the string – are bundled up into a tuple.

In [9]:
class Aircraft:

    def __init__(self, registration, model, num_rows, num_seats_per_row):
        self._registration = registration
        self._model = model
        self._num_rows = num_rows
        self._num_seats_per_row = num_seats_per_row
    
    def registration(self):
        return self._registration
    
    def model(self):
        return self._model
    
    def seating_plan(self):
        return (range(1, self._num_rows + 1), "ABCDEFGHJK"[:self._num_seats_per_row])

In [10]:
# construct a plane with a seating plan
a = Aircraft(" G-EUPT", "Airbus A319", num_rows = 22, num_seats_per_row = 6)

print(
    f"""
    Registration: {a.registration()}
    Model: {a.model()}
    Seating Plan: {a.seating_plan()}
    """
)


    Registration:  G-EUPT
    Model: Airbus A319
    Seating Plan: (range(1, 23), 'ABCDEF')
    


In [11]:
# test range object (should include 22 rows and 6 seats per row (letters))
row_seats = []

for item in a.seating_plan():
    for identifier in item:
        row_seats.append(identifier)

%pprint
row_seats

Pretty printing has been turned OFF


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 'A', 'B', 'C', 'D', 'E', 'F']

In [12]:
%pprint

Pretty printing has been turned ON


### Collaborating Classes
The Law of Demeter is an object-oriented design principle that says you should never call methods on objects you receive from other calls. Or, put another way: Only talk to your immediate friends.

We’ll now modify our Flight class to accept an aircraft object when it is constructed, and we’ll follow the Law of Demeter by adding a method to report the aircraft model. This method will delegate to Aircraft on behalf of the client rather than allowing the client to “reach through” the Flight and interrogate the Aircraft object directly.

Notice that we construct the Aircraft object and directly pass it to the Flight constructor without needing an intermediate named reference for it.

In [13]:
class Flight:
    """A Flight with a Particular Passenger Aircraft"""
    
    def __init__(self, flight_number, aircraft):
        if not flight_number[:2].isalpha():
            raise ValueError(f"No Airline Code in {flight_number}")
        
        if not flight_number[:2].isupper():
            raise ValueError(f"Invalid Airline Code {flight_number}")
        
        if not (flight_number[2:].isdigit() and int(flight_number[2:]) <= 9999):
            raise ValueError(f"Invalid Route Number {flight_number}")
        
        self._flight_number = flight_number
        self._aircraft = aircraft

    def flight_number(self):
        return self._flight_number
    
    def airline(self):
        return self._flight_number[:2]

    def aircraft_model(self):
        return self._aircraft.model()


class Aircraft:

    def __init__(self, registration, model, num_rows, num_seats_per_row):
        self._registration = registration
        self._model = model
        self._num_rows = num_rows
        self._num_seats_per_row = num_seats_per_row
    
    def registration(self):
        return self._registration
    
    def model(self):
        return self._model
    
    def seating_plan(self):
        return (range(1, self._num_rows + 1), "ABCDEFGHJK"[:self._num_seats_per_row])

In [14]:
# construct a flight with a specific aircraft
f = Flight("BA758", Aircraft("G-EUPT", "Airbus A319", num_rows = 22, num_seats_per_row = 6))

f.aircraft_model()

'Airbus A319'

### Booking Seats

We’ll represent the seat allocations using a list of dictionaries. The list will contain one entry for each seat row, and each entry will be a dictionary mapping from seat-letter to occupant name. If a seat is unoccupied, the corresponding dictionary value will contain None.

We'll add this logic to Flight.\_\_init\_\_()
- Line 17: Tuple unpacking to assign row and seat identifiers to local varoables (rows, seats)
- Line 18: List comprehension iterates over the rows object and concatenates the seat assignment to the seat mapping dictionary

In [15]:
class Flight:
    """A Flight with a Particular Passenger Aircraft"""
    
    def __init__(self, flight_number, aircraft):
        if not flight_number[:2].isalpha():
            raise ValueError(f"No Airline Code in {flight_number}")
        
        if not flight_number[:2].isupper():
            raise ValueError(f"Invalid Airline Code {flight_number}")
        
        if not (flight_number[2:].isdigit() and int(flight_number[2:]) <= 9999):
            raise ValueError(f"Invalid Route Number {flight_number}")
        
        self._flight_number = flight_number
        self._aircraft = aircraft
        
        rows, seats = self._aircraft.seating_plan()
        self._seating = [None] + [{letter: None for letter in seats} for _ in rows]

    def flight_number(self):
        return self._flight_number
    
    def airline(self):
        return self._flight_number[:2]

    def aircraft_model(self):
        return self._aircraft.model()


class Aircraft:

    def __init__(self, registration, model, num_rows, num_seats_per_row):
        self._registration = registration
        self._model = model
        self._num_rows = num_rows
        self._num_seats_per_row = num_seats_per_row
    
    def registration(self):
        return self._registration
    
    def model(self):
        return self._model
    
    def seating_plan(self):
        return (range(1, self._num_rows + 1), "ABCDEFGHJK"[:self._num_seats_per_row])

In [20]:
# test seat dictionary
f = Flight("BA758", Aircraft(" G-EUPT", "Airbus A319", num_rows = 22, num_seats_per_row = 6))
f._seating

[None,
 {'A': None, 'B': None, 'C': None, 'D': None, 'E': None, 'F': None},
 {'A': None, 'B': None, 'C': None, 'D': None, 'E': None, 'F': None},
 {'A': None, 'B': None, 'C': None, 'D': None, 'E': None, 'F': None},
 {'A': None, 'B': None, 'C': None, 'D': None, 'E': None, 'F': None},
 {'A': None, 'B': None, 'C': None, 'D': None, 'E': None, 'F': None},
 {'A': None, 'B': None, 'C': None, 'D': None, 'E': None, 'F': None},
 {'A': None, 'B': None, 'C': None, 'D': None, 'E': None, 'F': None},
 {'A': None, 'B': None, 'C': None, 'D': None, 'E': None, 'F': None},
 {'A': None, 'B': None, 'C': None, 'D': None, 'E': None, 'F': None},
 {'A': None, 'B': None, 'C': None, 'D': None, 'E': None, 'F': None},
 {'A': None, 'B': None, 'C': None, 'D': None, 'E': None, 'F': None},
 {'A': None, 'B': None, 'C': None, 'D': None, 'E': None, 'F': None},
 {'A': None, 'B': None, 'C': None, 'D': None, 'E': None, 'F': None},
 {'A': None, 'B': None, 'C': None, 'D': None, 'E': None, 'F': None},
 {'A': None, 'B': None, 'C'

### Allocating Seats to Passengers

- Line 6: Methods are functions, so deserve docstrings too. 
- Line 17: We get the seat letter by using negative indexing into the seat string. 
- Line 18: We test that the seat letter is valid by checking for membership of seat_letters using the in membership testing operator. 
- Line 21: We extract the row number using string slicing to take all but the last character. 
- Line 23: We try to convert the row number substring to an integer using the int() constructor. If this fails, we catch the ValueError and in the handler raise a new ValueError with a more appropriate message payload. 
- Line 27: We conveniently validate the row number by using the in operator against the rows object which is a range. We can do this because range() objects support the container protocol. 
- Line 30: We check that the requested seat is unoccupied using an identity test with None. If it’s occupied we raise a ValueError. 
- Line 33: If we get this far, everything is in good shape, and we can assign the seat.

In [22]:
class Flight:
    """A Flight with a Particular Passenger Aircraft"""
    
    def __init__(self, flight_number, aircraft):
        if not flight_number[:2].isalpha():
            raise ValueError(f"No Airline Code in {flight_number}")
        
        if not flight_number[:2].isupper():
            raise ValueError(f"Invalid Airline Code {flight_number}")
        
        if not (flight_number[2:].isdigit() and int(flight_number[2:]) <= 9999):
            raise ValueError(f"Invalid Route Number {flight_number}")
        
        self._flight_number = flight_number
        self._aircraft = aircraft
        
        rows, seats = self._aircraft.seating_plan()
        self._seating = [None] + [{letter: None for letter in seats} for _ in rows]

    def flight_number(self):
        return self._flight_number
    
    def airline(self):
        return self._flight_number[:2]

    def aircraft_model(self):
        return self._aircraft.model()
    
    def allocate_seat(self, seat, passenger):
        """Allocate a Seat to a Passenger.
        
        Args:
            seat: A seat designator such as '12C' or 21F'.
            passenger: The passenger name as a string.
        
        Raises:
            ValueError: If the seat is unavailable.
        """

        rows, seat_letters = self._aircraft.seating_plan()
        
        letter = seat[-1]
        if letter not in seat_letters:
            raise ValueError(f"Invalid Seat Letter {letter}")
        
        row_text = seat[:-1]
        try:
            row = int(row_text)
        except ValueError:
            raise ValueError(f"Invalid Seat Row {row_text}")
        
        if row not in rows:
            raise ValueError(f"Invalid Row Number {row}")
        
        if self._seating[row][letter] is not None:
            raise ValueError(f"Seat {seat} Already Occupied")
        
        self._seating[row][letter] = passenger


class Aircraft:

    def __init__(self, registration, model, num_rows, num_seats_per_row):
        self._registration = registration
        self._model = model
        self._num_rows = num_rows
        self._num_seats_per_row = num_seats_per_row
    
    def registration(self):
        return self._registration
    
    def model(self):
        return self._model
    
    def seating_plan(self):
        return (range(1, self._num_rows + 1), "ABCDEFGHJK"[:self._num_seats_per_row])

In [52]:
# test seat assignment
f = Flight("BA758", Aircraft(" G-EUPT", "Airbus A319", num_rows = 22, num_seats_per_row = 6))
f.allocate_seat('5A', 'Guido van Rossum')
f.allocate_seat('6F', 'Bjarne Stroustrup')
f.allocate_seat('6E', 'Anders Hejlsberg')
f.allocate_seat('1C', 'John McCarthy')
f.allocate_seat('1D', 'Richard Hickey')

# f.allocate_seat('12A', 'Rasmus Lerdorf') # raises ValueError: Seat 12A Already Occupied
# f.allocate_seat('E27', 'Yukihiro Matsumoto') # raises ValueError: Invalid Seat Letter 7
# f.allocate_seat('DD', 'Larry Wall') # raises ValueError: Invalid Seat Row D

f._seating

[None,
 {'A': None,
  'B': None,
  'C': 'John McCarthy',
  'D': 'Richard Hickey',
  'E': None,
  'F': None},
 {'A': None, 'B': None, 'C': None, 'D': None, 'E': None, 'F': None},
 {'A': None, 'B': None, 'C': None, 'D': None, 'E': None, 'F': None},
 {'A': None, 'B': None, 'C': None, 'D': None, 'E': None, 'F': None},
 {'A': 'Guido van Rossum',
  'B': None,
  'C': None,
  'D': None,
  'E': None,
  'F': None},
 {'A': None,
  'B': None,
  'C': None,
  'D': None,
  'E': 'Anders Hejlsberg',
  'F': 'Bjarne Stroustrup'},
 {'A': None, 'B': None, 'C': None, 'D': None, 'E': None, 'F': None},
 {'A': None, 'B': None, 'C': None, 'D': None, 'E': None, 'F': None},
 {'A': None, 'B': None, 'C': None, 'D': None, 'E': None, 'F': None},
 {'A': None, 'B': None, 'C': None, 'D': None, 'E': None, 'F': None},
 {'A': None, 'B': None, 'C': None, 'D': None, 'E': None, 'F': None},
 {'A': None, 'B': None, 'C': None, 'D': None, 'E': None, 'F': None},
 {'A': None, 'B': None, 'C': None, 'D': None, 'E': None, 'F': None},


In [35]:
# refactor seat designator parsing and validation into its own method (_parse_seat())
class Flight:
    """A Flight with a Particular Passenger Aircraft"""
    
    def __init__(self, flight_number, aircraft):
        if not flight_number[:2].isalpha():
            raise ValueError(f"No Airline Code in {flight_number}")
        
        if not flight_number[:2].isupper():
            raise ValueError(f"Invalid Airline Code {flight_number}")
        
        if not (flight_number[2:].isdigit() and int(flight_number[2:]) <= 9999):
            raise ValueError(f"Invalid Route Number {flight_number}")
        
        self._flight_number = flight_number
        self._aircraft = aircraft
        
        rows, seats = self._aircraft.seating_plan()
        self._seating = [None] + [{letter: None for letter in seats} for _ in rows]

    def flight_number(self):
        return self._flight_number
    
    def airline(self):
        return self._flight_number[:2]

    def aircraft_model(self):
        return self._aircraft.model()
    
    def allocate_seat(self, seat, passenger):
        """Allocate a Seat to a Passenger.
        
        Args:
            seat: A seat designator such as '12C' or 21F'.
            passenger: The passenger name as a string.
        
        Raises:
            ValueError: If the seat is unavailable.
        """
        row, letter = self._parse_seat(seat)
        
        if self._seating[row][letter] is not None:
            raise ValueError(f"Seat {seat} Already Occupied")
        
        self._seating[row][letter] = passenger

    def _parse_seat(self, seat):
        """Parse a Seat Designator into a Valid Row & Letter
        
        Args:
            seat: A seat designator such as '12F'.
        
        Returns:
            A tuple containing an integer and a string for row and seat
        """
        row_numbers, seat_letters = self._aircraft.seating_plan()

        letter = seat[-1]
        if letter not in seat_letters:
            raise ValueError(f"Invalid Seat Letter {letter}")
        
        row_text = seat[:-1]
        try:
            row = int(row_text)
        except ValueError:
            raise ValueError(f"Invalid Seat Row {row}")
        
        if row not in row_numbers:
            raise ValueError(f"Invalid Row Number {row}")
        
        return row, letter


class Aircraft:

    def __init__(self, registration, model, num_rows, num_seats_per_row):
        self._registration = registration
        self._model = model
        self._num_rows = num_rows
        self._num_seats_per_row = num_seats_per_row
    
    def registration(self):
        return self._registration
    
    def model(self):
        return self._model
    
    def seating_plan(self):
        return (range(1, self._num_rows + 1), "ABCDEFGHJK"[:self._num_seats_per_row])

### Implement relocate_passenger() Method & Add Convenience Function

- Line 72: This parses and validates the from_seat and to_seat arguments and then moves the passenger to the new location.
- Line 110: Add a module level convenience function to create a flight object

In [54]:
class Flight:
    """A Flight with a Particular Passenger Aircraft"""
    
    def __init__(self, flight_number, aircraft):
        if not flight_number[:2].isalpha():
            raise ValueError(f"No Airline Code in {flight_number}")
        
        if not flight_number[:2].isupper():
            raise ValueError(f"Invalid Airline Code {flight_number}")
        
        if not (flight_number[2:].isdigit() and int(flight_number[2:]) <= 9999):
            raise ValueError(f"Invalid Route Number {flight_number}")
        
        self._flight_number = flight_number
        self._aircraft = aircraft
        
        rows, seats = self._aircraft.seating_plan()
        self._seating = [None] + [{letter: None for letter in seats} for _ in rows]

    def flight_number(self):
        return self._flight_number
    
    def airline(self):
        return self._flight_number[:2]

    def aircraft_model(self):
        return self._aircraft.model()
    
    def allocate_seat(self, seat, passenger):
        """Allocate a Seat to a Passenger.
        
        Args:
            seat: A seat designator such as '12C' or 21F'.
            passenger: The passenger name as a string.
        
        Raises:
            ValueError: If the seat is unavailable.
        """
        row, letter = self._parse_seat(seat)
        
        if self._seating[row][letter] is not None:
            raise ValueError(f"Seat {seat} Already Occupied")
        
        self._seating[row][letter] = passenger

    def _parse_seat(self, seat):
        """Parse a Seat Designator into a Valid Row & Letter
        
        Args:
            seat: A seat designator such as '12F'.
        
        Returns:
            A tuple containing an integer and a string for row and seat
        """
        row_numbers, seat_letters = self._aircraft.seating_plan()

        letter = seat[-1]
        if letter not in seat_letters:
            raise ValueError(f"Invalid Seat Letter {letter}")
        
        row_text = seat[:-1]
        try:
            row = int(row_text)
        except ValueError:
            raise ValueError(f"Invalid Seat Row {row}")
        
        if row not in row_numbers:
            raise ValueError(f"Invalid Row Number {row}")
        
        return row, letter
    
    def relocate_passenger(self, from_seat, to_seat):
        """Relocate a passenger to a different seat
        
        Args:
            from_seat: The existing seat designator for the passenger to be moved.
        
            to_seat: The new seat designator.
        """
        from_row, from_letter = self._parse_seat(from_seat)
        if self._seating[from_row][from_letter] is None:
            raise ValueError(f"No Passenger to Relocate from Seat {from_seat}")
        
        to_row, to_letter = self._parse_seat(to_seat)
        if self._seating[to_row][to_letter] is not None:
            raise ValueError(f"Seat {to_seat} Already Occupied")
        
        self._seating[to_row][to_letter] = self._seating[from_row][from_letter]
        self._seating[from_row][from_letter] = None


class Aircraft:

    def __init__(self, registration, model, num_rows, num_seats_per_row):
        self._registration = registration
        self._model = model
        self._num_rows = num_rows
        self._num_seats_per_row = num_seats_per_row
    
    def registration(self):
        return self._registration
    
    def model(self):
        return self._model
    
    def seating_plan(self):
        return (range(1, self._num_rows + 1), "ABCDEFGHJK"[:self._num_seats_per_row])


def make_flight():
    f = Flight("BA758", Aircraft("G-EUPT", "Airbus A319", num_rows = 22, num_seats_per_row = 6))
    f.allocate_seat('5A', 'Guido van Rossum')
    f.allocate_seat('6F', 'Bjarne Stroustrup')
    f.allocate_seat('6E', 'Anders Hejlsberg')
    f.allocate_seat('1C', 'John McCarthy')
    f.allocate_seat('1D', 'Richard Hickey')

    return f

In [55]:
# test seat reallocation
f = make_flight()

f.relocate_passenger('5A', '6D')
f._seating

[None,
 {'A': None,
  'B': None,
  'C': 'John McCarthy',
  'D': 'Richard Hickey',
  'E': None,
  'F': None},
 {'A': None, 'B': None, 'C': None, 'D': None, 'E': None, 'F': None},
 {'A': None, 'B': None, 'C': None, 'D': None, 'E': None, 'F': None},
 {'A': None, 'B': None, 'C': None, 'D': None, 'E': None, 'F': None},
 {'A': None, 'B': None, 'C': None, 'D': None, 'E': None, 'F': None},
 {'A': None,
  'B': None,
  'C': None,
  'D': 'Guido van Rossum',
  'E': 'Anders Hejlsberg',
  'F': 'Bjarne Stroustrup'},
 {'A': None, 'B': None, 'C': None, 'D': None, 'E': None, 'F': None},
 {'A': None, 'B': None, 'C': None, 'D': None, 'E': None, 'F': None},
 {'A': None, 'B': None, 'C': None, 'D': None, 'E': None, 'F': None},
 {'A': None, 'B': None, 'C': None, 'D': None, 'E': None, 'F': None},
 {'A': None, 'B': None, 'C': None, 'D': None, 'E': None, 'F': None},
 {'A': None, 'B': None, 'C': None, 'D': None, 'E': None, 'F': None},
 {'A': None, 'B': None, 'C': None, 'D': None, 'E': None, 'F': None},
 {'A': Non

### Counting Available Seats

- Create num_available_seats() method to count number of available seats
- Two nested generator expressions
    - The outer expression filters for all rows which are not None to exclude our dummy first row. The value of each item in the outer expression is the sum of the number of None values in each row.
    - The inner expression iterates over values of the dictionary and adds 1 for each None found.

In [57]:
class Flight:
    """A Flight with a Particular Passenger Aircraft"""
    
    def __init__(self, flight_number, aircraft):
        if not flight_number[:2].isalpha():
            raise ValueError(f"No Airline Code in {flight_number}")
        
        if not flight_number[:2].isupper():
            raise ValueError(f"Invalid Airline Code {flight_number}")
        
        if not (flight_number[2:].isdigit() and int(flight_number[2:]) <= 9999):
            raise ValueError(f"Invalid Route Number {flight_number}")
        
        self._flight_number = flight_number
        self._aircraft = aircraft
        
        rows, seats = self._aircraft.seating_plan()
        self._seating = [None] + [{letter: None for letter in seats} for _ in rows]

    def flight_number(self):
        return self._flight_number
    
    def airline(self):
        return self._flight_number[:2]

    def aircraft_model(self):
        return self._aircraft.model()
    
    def allocate_seat(self, seat, passenger):
        """Allocate a Seat to a Passenger.
        
        Args:
            seat: A seat designator such as '12C' or 21F'.
            passenger: The passenger name as a string.
        
        Raises:
            ValueError: If the seat is unavailable.
        """
        row, letter = self._parse_seat(seat)
        
        if self._seating[row][letter] is not None:
            raise ValueError(f"Seat {seat} Already Occupied")
        
        self._seating[row][letter] = passenger

    def _parse_seat(self, seat):
        """Parse a Seat Designator into a Valid Row & Letter
        
        Args:
            seat: A seat designator such as '12F'.
        
        Returns:
            A tuple containing an integer and a string for row and seat
        """
        row_numbers, seat_letters = self._aircraft.seating_plan()

        letter = seat[-1]
        if letter not in seat_letters:
            raise ValueError(f"Invalid Seat Letter {letter}")
        
        row_text = seat[:-1]
        try:
            row = int(row_text)
        except ValueError:
            raise ValueError(f"Invalid Seat Row {row}")
        
        if row not in row_numbers:
            raise ValueError(f"Invalid Row Number {row}")
        
        return row, letter
    
    def relocate_passenger(self, from_seat, to_seat):
        """Relocate a passenger to a different seat
        
        Args:
            from_seat: The existing seat designator for the passenger to be moved.
        
            to_seat: The new seat designator.
        """
        from_row, from_letter = self._parse_seat(from_seat)
        if self._seating[from_row][from_letter] is None:
            raise ValueError(f"No Passenger to Relocate from Seat {from_seat}")
        
        to_row, to_letter = self._parse_seat(to_seat)
        if self._seating[to_row][to_letter] is not None:
            raise ValueError(f"Seat {to_seat} Already Occupied")
        
        self._seating[to_row][to_letter] = self._seating[from_row][from_letter]
        self._seating[from_row][from_letter] = None

    def num_available_seats(self):
        return sum(sum(1 for s in row.values() if s is None)
            for row in self._seating
            if row is not None)

class Aircraft:

    def __init__(self, registration, model, num_rows, num_seats_per_row):
        self._registration = registration
        self._model = model
        self._num_rows = num_rows
        self._num_seats_per_row = num_seats_per_row
    
    def registration(self):
        return self._registration
    
    def model(self):
        return self._model
    
    def seating_plan(self):
        return (range(1, self._num_rows + 1), "ABCDEFGHJK"[:self._num_seats_per_row])


def make_flight():
    f = Flight("BA758", Aircraft("G-EUPT", "Airbus A319", num_rows = 22, num_seats_per_row = 6))
    f.allocate_seat('5A', 'Guido van Rossum')
    f.allocate_seat('6F', 'Bjarne Stroustrup')
    f.allocate_seat('6E', 'Anders Hejlsberg')
    f.allocate_seat('1C', 'John McCarthy')
    f.allocate_seat('1D', 'Richard Hickey')

    return f

In [58]:
# test available seat counter
f = make_flight()
f.num_available_seats() # 132 total seats avaiable - 5 assigned seats = 127

127

 # Add Module Function to Print Boarding Pass

- Line 96 & 100: This tells the card_printer to print each passenger, having sorted a list of passenger-seat tuples obtained from a _passenger_seats() implementation detail method (note the leading underscore).
- Line 119: We measure the length of this output line, build some banners and borders around it, and concatenate the lines together using the join() method called on a newline separator. The whole card is then printed, followed by a blank line. The card printer doesn’t know anything about Flights or Aircraft – it’s very loosely coupled.

In [5]:
class Flight:
    """A Flight with a Particular Passenger Aircraft"""
    
    def __init__(self, flight_number, aircraft):
        if not flight_number[:2].isalpha():
            raise ValueError(f"No Airline Code in {flight_number}")
        
        if not flight_number[:2].isupper():
            raise ValueError(f"Invalid Airline Code {flight_number}")
        
        if not (flight_number[2:].isdigit() and int(flight_number[2:]) <= 9999):
            raise ValueError(f"Invalid Route Number {flight_number}")
        
        self._flight_number = flight_number
        self._aircraft = aircraft
        
        rows, seats = self._aircraft.seating_plan()
        self._seating = [None] + [{letter: None for letter in seats} for _ in rows]

    def flight_number(self):
        return self._flight_number
    
    def airline(self):
        return self._flight_number[:2]

    def aircraft_model(self):
        return self._aircraft.model()
    
    def allocate_seat(self, seat, passenger):
        """Allocate a Seat to a Passenger.
        
        Args:
            seat: A seat designator such as '12C' or 21F'.
            passenger: The passenger name as a string.
        
        Raises:
            ValueError: If the seat is unavailable.
        """
        row, letter = self._parse_seat(seat)
        
        if self._seating[row][letter] is not None:
            raise ValueError(f"Seat {seat} Already Occupied")
        
        self._seating[row][letter] = passenger

    def _parse_seat(self, seat):
        """Parse a Seat Designator into a Valid Row & Letter
        
        Args:
            seat: A seat designator such as '12F'.
        
        Returns:
            A tuple containing an integer and a string for row and seat
        """
        row_numbers, seat_letters = self._aircraft.seating_plan()

        letter = seat[-1]
        if letter not in seat_letters:
            raise ValueError(f"Invalid Seat Letter {letter}")
        
        row_text = seat[:-1]
        try:
            row = int(row_text)
        except ValueError:
            raise ValueError(f"Invalid Seat Row {row}")
        
        if row not in row_numbers:
            raise ValueError(f"Invalid Row Number {row}")
        
        return row, letter
    
    def relocate_passenger(self, from_seat, to_seat):
        """Relocate a passenger to a different seat
        
        Args:
            from_seat: The existing seat designator for the passenger to be moved.
        
            to_seat: The new seat designator.
        """
        from_row, from_letter = self._parse_seat(from_seat)
        if self._seating[from_row][from_letter] is None:
            raise ValueError(f"No Passenger to Relocate from Seat {from_seat}")
        
        to_row, to_letter = self._parse_seat(to_seat)
        if self._seating[to_row][to_letter] is not None:
            raise ValueError(f"Seat {to_seat} Already Occupied")
        
        self._seating[to_row][to_letter] = self._seating[from_row][from_letter]
        self._seating[from_row][from_letter] = None

    def num_available_seats(self):
        return sum(sum(1 for s in row.values() if s is None)
            for row in self._seating
            if row is not None)

    def make_boarding_cards(self, card_printer):
        for passenger, seat in sorted(self._passenger_seats()):
            card_printer(passenger, seat, self.flight_number(), self.aircraft_model())

    def _passenger_seats(self):
        """An iterable series of passenger seating locations."""
        row_numbers, seat_letters = self._aircraft.seating_plan()
        for row in row_numbers:
            for letter in seat_letters:
                passenger = self._seating[row][letter]
                if passenger is not None:
                    yield (passenger, f"{row}{letter}")
                    
            
class Aircraft:

    def __init__(self, registration, model, num_rows, num_seats_per_row):
        self._registration = registration
        self._model = model
        self._num_rows = num_rows
        self._num_seats_per_row = num_seats_per_row
    
    def registration(self):
        return self._registration
    
    def model(self):
        return self._model
    
    def seating_plan(self):
        return (range(1, self._num_rows + 1), "ABCDEFGHJK"[:self._num_seats_per_row])


def console_card_printer(passenger, seat, flight_number, aircraft):
    output = f"| Name: {passenger}"         \
             f"  Flight: {flight_number}"   \
             f"  Seat: {seat}"              \
             f" Aircraft: {aircraft}"       \
             " |"
    
    banner = "+" + "-" * (len(output) -2) + "+"
    border = "|" + " " * (len(output) -2) + "|"
    lines = [banner, border, output, border, banner]
    card = "\n".join(lines)

    print(card)
    print()   


def make_flight():
    f = Flight("BA758", Aircraft("G-EUPT", "Airbus A319", num_rows = 22, num_seats_per_row = 6))
    f.allocate_seat('5A', 'Guido van Rossum')
    f.allocate_seat('6F', 'Bjarne Stroustrup')
    f.allocate_seat('6E', 'Anders Hejlsberg')
    f.allocate_seat('1C', 'John McCarthy')
    f.allocate_seat('1D', 'Richard Hickey')

    return f

In [6]:
# test boarding pass
f = make_flight()
f.make_boarding_cards(console_card_printer)

+-----------------------------------------------------------------------+
|                                                                       |
| Name: Anders Hejlsberg  Flight: BA758  Seat: 6E Aircraft: Airbus A319 |
|                                                                       |
+-----------------------------------------------------------------------+

+------------------------------------------------------------------------+
|                                                                        |
| Name: Bjarne Stroustrup  Flight: BA758  Seat: 6F Aircraft: Airbus A319 |
|                                                                        |
+------------------------------------------------------------------------+

+-----------------------------------------------------------------------+
|                                                                       |
| Name: Guido van Rossum  Flight: BA758  Seat: 5A Aircraft: Airbus A319 |
|                              

### Refactoring

The design of the Aircraft class is somewhat flawed because users who instantiate it have to supply a seating configuration that matches the aircraft model. For the purposes of this exercise we can assume that the seating arrangement is fixed per aircraft model. Better, and simpler, make separate derived classes for each specific model of aircraft with a fixed seating configuration.

The base Aircraft class can now be used for common elements of aircraft types to prevent code duplication. The individual aircraft type classes will use inheritance to access the base class. These derived classes only contain the specifics for that aircraft type. All general functionality is shared from the base class by inheritance.

We also update the make_flight() function to accept this refactoring.

In [15]:
class Flight:
    """A flight with a particular passenger aircraft."""

    def __init__(self, flight_number, aircraft):
        if not flight_number[:2].isalpha():
            raise ValueError(f"No Airline Code in '{flight_number}'")
        
        if not flight_number[:2].isupper():
            raise ValueError(f"Invalid Airline Code '{flight_number}'")
        
        if not (flight_number[2:].isdigit() and int(flight_number[2:]) <= 9999):
            raise ValueError(f"Invalid Route Number '{flight_number}'")

        self._flight_number = flight_number
        self._aircraft = aircraft
        
        rows, seats = self._aircraft.seating_plan()
        self._seating = [None] + [{letter: None for letter in seats} for _ in rows]
    
    def aircraft_model(self):
        return self._aircraft.model()
    
    def flight_number(self):
        return self._flight_number
    
    def airline(self):
        return self._flight_number[:2]

    def allocate_seat(self, seat, passenger):
        """Allocate a seat to a passenger.
        
        Args:
            seat: A seat designator such as '12C' or '21F'.
            passenger: The passenger name.
        
        Raises:
            ValueError: If the seat is unavailable.
        """
        row, letter = self._parse_seat(seat)
        
        if self._seating[row][letter] is not None:
            raise ValueError(f"Seat {seat} Already Occupied")
        
        self._seating[row][letter] = passenger
    
    def _parse_seat(self, seat):
        rows, seat_letters = self._aircraft.seating_plan()

        letter = seat[-1]
        if letter not in seat_letters:
            raise ValueError(f"Invalid Seat Letter {letter}")
        
        row_text = seat[:-1]
        try:
            row = int(row_text)
        except ValueError:
            raise ValueError(f"Invalid Seat Row {row}")
        
        if row not in rows:
            raise ValueError(f"Invalid Row Number {row}")
        
        return row, letter

    def relocate_passenger(self, from_seat, to_seat):
        """Relocate a passenger to a different seat
        
        Args:
            from_seat: The existing seat designator for the passenger to be moved.
        
            to_seat: The new seat designator.
        """
        from_row, from_letter = self._parse_seat(from_seat)
        if self._seating[from_row][from_letter] is None:
            raise ValueError(f"No Passenger to Relocate from Seat {from_seat}")
        
        to_row, to_letter = self._parse_seat(to_seat)
        if self._seating[to_row][to_letter] is not None:
            raise ValueError(f"Seat {to_seat} Already Occupied")
        
        self._seating[to_row][to_letter] = self._seating[from_row][from_letter]
        self._seating[from_row][from_letter] = None
    
    def num_available_seats(self):
        return sum(sum(1 for s in row.values() if s is None)
            for row in self._seating
            if row is not None)
    
    def make_boarding_cards(self, card_printer):
        for passenger, seat in sorted(self._passenger_seats()):
            card_printer(passenger, seat, self.flight_number(), self.aircraft_model())
    
    def _passenger_seats(self):
        """An iterable series of passenger seating locations."""
        row_numbers, seat_letters = self._aircraft.seating_plan()
        for row in row_numbers:
            for letter in seat_letters:
                passenger = self._seating[row][letter]
                if passenger is not None:
                    yield (passenger, f"{row}{letter}")


class Aircraft:

    def __init__(self, registration):
        self._registration = registration
    
    def registration(self):
        return self._registration
        
    def num_seats(self):
        rows, row_seats = self.seating_plan()
        return len(rows) * len(row_seats)


class AirbusA319(Aircraft):
    
    def model(self):
        return "Airbus A319"
    
    def seating_plan(self):
        return range(1, 23), "ABCDEF"


class Boeing777(Aircraft):
    
    def model(self):
        return "Boeing 777"
    
    def seating_plan(self):
        return range(1, 56), "ABCDEGHJK"


# module function
def console_card_printer(passenger, seat, flight_number, aircraft):
    output = f"| Name: {passenger}"         \
             f"  Flight: {flight_number}"   \
             f"  Seat: {seat}"              \
             f" Aircraft: {aircraft}"       \
             "|"
    
    banner = "+" + "-" * (len(output) -2) + "+"
    border = "|" + " " * (len(output) -2) + "|"
    lines = [banner, border, output, border, banner]
    card = "\n".join(lines)

    print(card)
    print()            


# convenience function for testing
def make_flights():

    f = Flight("BA758", AirbusA319("G-EUPT"))
    f.allocate_seat("12A", "Guido van Rossum")
    f.allocate_seat("15F", "Bjarne Stroustrup")
    f.allocate_seat("15E", "Anders Hejlsberg")
    f.allocate_seat("1C", "John McCarthy")
    f.allocate_seat("1D", "Rich Hickey")

    g = Flight("AF72", Boeing777("F-GSPS"))
    g.allocate_seat("55K", "Larry Wall")
    g.allocate_seat("33G", "Yukihiro Matsumoto")
    g.allocate_seat("4B", "Brian Kernighan")
    g.allocate_seat("4A", "John McCarthy")
    g.allocate_seat("1D", "Dennis Ritchie")

    return f, g

In [16]:
# testing refactor and printing boarding cards for f
f, g = make_flights()
print(f.make_boarding_cards(console_card_printer))

+-----------------------------------------------------------------------+
|                                                                       |
| Name: Anders Hejlsberg  Flight: BA758  Seat: 15E Aircraft: Airbus A319|
|                                                                       |
+-----------------------------------------------------------------------+

+------------------------------------------------------------------------+
|                                                                        |
| Name: Bjarne Stroustrup  Flight: BA758  Seat: 15F Aircraft: Airbus A319|
|                                                                        |
+------------------------------------------------------------------------+

+-----------------------------------------------------------------------+
|                                                                       |
| Name: Guido van Rossum  Flight: BA758  Seat: 12A Aircraft: Airbus A319|
|                              

In [17]:
# testing refactor and printing boarding cards for g
f, g = make_flights()
print(g.make_boarding_cards(console_card_printer))

+-------------------------------------------------------------------+
|                                                                   |
| Name: Brian Kernighan  Flight: AF72  Seat: 4B Aircraft: Boeing 777|
|                                                                   |
+-------------------------------------------------------------------+

+------------------------------------------------------------------+
|                                                                  |
| Name: Dennis Ritchie  Flight: AF72  Seat: 1D Aircraft: Boeing 777|
|                                                                  |
+------------------------------------------------------------------+

+-----------------------------------------------------------------+
|                                                                 |
| Name: John McCarthy  Flight: AF72  Seat: 4A Aircraft: Boeing 777|
|                                                                 |
+------------------------------

In [10]:
# testing num seats
a = AirbusA319("G-EZBT")
a.num_seats()

132

In [11]:
# testing num seats
b = Boeing777(" N717AN")
b.num_seats()

495

### Summary
- All types in Python have a ‘class’. 
- Classes define the structure and behavior of an object. 
- The class of an object is determined when the object is created and is almost always fixed for the lifetime of the object. 
- Classes are the key support for Object-Oriented Programming in Python. 
- Classes are defined using the class keyword followed by the class name, which is in CamelCase. 
- Instances of a class are created by calling the class as if it were a function. 
- Instance methods are functions defined inside the class which should accept an object instance called self as the first parameter. 
- Methods are called using the instance.method() syntax which is syntactic sugar for passing the instance as the formal self argument to the method. 
- An optional special initializer method called __init__() can be provided which is used to configure the self object at creation time. 
- The constructor calls the __init__() method if one is present. 
- The __init__() method is not the constructor. The object has been already constructed by the time the initializer is called. The initializer configures the newly created object before it’s returned to the caller of the constructor. 
- Arguments passed to the constructor are forwarded to the initializer. 
- Instance attributes are brought into existence by assigning to them. 
- Attributes and methods which are implementation details are by convention prefixed with an underscore. There are no public, protected or private access modifiers in Python. 
- Access to implementation details from outside the class can be very useful during development, testing and debugging. 
- Class invariants should be established in the initializer. If the invariants can’t be established raise exceptions to signal failure. 
- Methods can have docstrings, just like regular functions. 
- Classes can have docstrings. 
- Even within an object method calls must be qualified with self. 
- You can have as many classes and functions in a module as you wish. Related classes and global functions are usually grouped together this way. 
- Polymorphism in Python is achieved through duck typing where attributes and methods are only resolved at point of use - a behaviour called late-binding. 
- Polymorphism in Python does not require shared base classes or named interfaces. 
- Class inheritance in Python is primarily useful for sharing implementation rather than being necessary for polymorphism. 
- All methods are inherited, including special methods like the initializer.

Along the way we found out;
- Strings support slicing, because they implement the sequence protocol. 
- Following the Law of Demeter can reduce coupling.
- We can nest comprehensions. 
- It can sometimes be useful to discard the current item in a comprehension using a dummy reference, conventionally the underscore. 
- When dealing with one-based collections it’s often easier just to waste the zeroth list entry. 
- Don’t feel compelled to use classes when a simple function will suffice. Functions are also objects. 
- Complex comprehensions or generator expressions can be split over multiple lines to aid readability. 
- Statements can be split over multiple lines using the backslash line continuation character. Use this feature sparingly and only when it improves readability.
- Object-oriented design where one object tells another information can be more loosely coupled than those where one object queries another. “Tell! Don’t ask.”